In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import roc_curve, auc, classification_report,accuracy_score,precision_score,f1_score,recall_score,cohen_kappa_score,multilabel_confusion_matrix,make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import metrics
import joblib
from scipy import sparse
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.model_selection import cross_val_score,cross_validate

In [2]:
# load data 
dataframe = pd.read_csv(r'C:\Users\dawns\Desktop\Master thesis\data\Features_level.csv')
#data=dataframe[['CONTENT_NEW','','SCORE']].copy()
df_data=dataframe[['CONTENT_NEW','WORDCUT','SCORE','LEVEL','Tokens','level1_tokens','level2_tokens','level3_tokens','level4_tokens','level5_tokens','level6_tokens','level7_tokens','char_num','high_level','primary_level','Connective_ratio','connective_SCORE','pre','connective_numbers']].copy()


In [3]:
# Generating 'y' for classification as well as regression
y = df_data['SCORE']

In [4]:

# generating x 
pre = df_data['pre']
# Tokenization
def preprocessing_sentence(x):
    words = jieba.cut(str(x).strip())
    return ' '.join(words)

In [5]:
#ngrams Vectorizer
print(pre.shape)
cv = CountVectorizer(ngram_range=(1, 3), max_features=1000)
ngram_vector = cv.fit_transform(pre)
print(ngram_vector.shape)


(3668,)
(3668, 1000)


In [6]:
# 特征合并 feature concatenate 
feature_set1 = df_data[['connective_SCORE',]]
feature_set2 = ngram_vector
print(feature_set1.shape)
print(feature_set2.shape)
train_features = sparse.hstack((feature_set2, feature_set1))
print('=============')
print(train_features.shape)

(3668, 1)
(3668, 1000)
(3668, 1001)


In [7]:
# 划分训练集 ，测试集 train_test split
x_train, x_test, y_train, y_test = train_test_split(feature_set2, y, test_size=0.2)
print(x_train.shape)
print(y_train.shape)

(2934, 1000)
(2934,)


In [8]:
#Tf-IDF + LR 
#评估
# （准确率、精确度、召回率、F1值、Kappa系数用于评估分类任务）
## 二分类问题，一般设为两类，正类/反类，或0、1. 所以有tp，fn，fp，tn。 t，f代表是否正确，p，n代表是正类还是反类
## 准确率（accuracy）：所有分类正确的样本数，除以样本总数）A= （tp+tn）/（tp+tn+fp+fn）
## 精确度（precision）精确度也叫查准率，判为正的为p，包括正确tp，和错误fp，其中tp即预测正确的比例 P= tp/tp+fp
## 召回率（recall） ： 查全率，判断正确的且为正的是tp，判断错误且为负（该正却负的），想知道tp占实际正确的比例 r= tp/tp+fn
## f1 越大越好，f1=2*tp/(2*tp+fp+fn)
## kappa 衡量分类精度，其计算基于混淆矩阵，目前用的是二次加权kappa，对错误的惩罚更重，应用更广泛
# 逻辑回归LR
lr = LogisticRegression()
lr.fit(x_train, y_train)
train_score = lr.score(x_train, y_train)
y_pred = lr.predict(x_test)
print("Train Accuracy:", train_score)
print("================================================")
y_pred = lr.predict(x_test)
test_score = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)


Train Accuracy: 0.9815950920245399
Test Accuracy: 0.20027247956403268
acc_for_each_class:
 [0.         0.         0.11111111 0.1875     0.20224719 0.16666667
 0.30322581 0.16666667 0.15384615 0.16       0.22916667 0.2       ]
qwkappa:
 0.4758248703316602
Weighted precision 0.203373097380379
Weighted recall 0.20027247956403268
Weighted f1-score 0.19924583323739337
classification report: 
               precision    recall  f1-score   support

          40       0.00      0.00      0.00         2
          45       0.00      0.00      0.00         7
          50       0.11      0.11      0.11        18
          55       0.19      0.11      0.13        57
          60       0.20      0.20      0.20        90
          65       0.17      0.16      0.16        98
          70       0.30      0.29      0.30       162
          75       0.17      0.19      0.18        90
          80       0.15      0.20      0.17        80
          85       0.16      0.24      0.19        51
          90  

d:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
print(y_pred.min())
print(y_pred.max())

40
95


In [10]:
# randomforest classfier
rf = ensemble.RandomForestClassifier()
params = {'n_estimators': [50], 'max_depth':[100], 'max_features':[10]}
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
train_score = rf.score(x_train, y_train)
test_score = accuracy_score(y_test, y_pred)
print("Train Accuracy:", train_score)
print("================================================")
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)



Train Accuracy: 0.9996591683708248
Test Accuracy: 0.25885558583106266
acc_for_each_class:
 [0.         0.         0.         0.1        0.24590164 0.13934426
 0.30097087 0.23529412 0.17948718 0.23333333 0.43478261 1.        ]
qwkappa:
 0.3754489990136851
Weighted precision 0.25131092703291763
Weighted recall 0.25885558583106266
Weighted f1-score 0.2053450945073627
classification report: 
               precision    recall  f1-score   support

          40       0.00      0.00      0.00         2
          45       0.00      0.00      0.00         7
          50       0.00      0.00      0.00        18
          55       0.10      0.02      0.03        57
          60       0.25      0.17      0.20        90
          65       0.14      0.17      0.15        98
          70       0.30      0.77      0.43       162
          75       0.24      0.09      0.13        90
          80       0.18      0.09      0.12        80
          85       0.23      0.14      0.17        51
          90 

d:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are

In [11]:
# support vector machine classfier
model = OneVsRestClassifier(svm.SVC(kernel='rbf',gamma='scale'))#linear
clt = model.fit(x_train,y_train)
y_pred = clt.predict(x_test)
test_score = accuracy_score(y_test, y_pred)
train_score = clt.score(x_train, y_train)
print("Train Accuracy:", train_score)
print("================================================")
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)

Train Accuracy: 0.994546693933197
Test Accuracy: 0.25204359673024523
acc_for_each_class:
 [0.         0.2        0.         0.20833333 0.21794872 0.26190476
 0.32432432 0.19402985 0.1509434  0.10204082 0.27272727 0.375     ]
qwkappa:
 0.44188214503406875
Weighted precision 0.23111074423157604
Weighted recall 0.25204359673024523
Weighted f1-score 0.2279554996699112
classification report: 
               precision    recall  f1-score   support

          40       0.00      0.00      0.00         2
          45       0.20      0.14      0.17         7
          50       0.00      0.00      0.00        18
          55       0.21      0.09      0.12        57
          60       0.22      0.19      0.20        90
          65       0.26      0.22      0.24        98
          70       0.32      0.59      0.42       162
          75       0.19      0.14      0.17        90
          80       0.15      0.10      0.12        80
          85       0.10      0.10      0.10        51
          90 

In [12]:
# 交叉验证 cross validation   logistic regression
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(lr, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

d:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

{'fit_time': array([0.95443892, 1.08485031, 0.96374011, 0.96272326, 0.96117592,
       0.98288822, 0.93649101, 0.91806769, 0.9155252 , 0.90408349]), 'score_time': array([0.00299788, 0.0009985 , 0.00199962, 0.00243998, 0.00099969,
       0.00102806, 0.00200105, 0.00200129, 0.00103092, 0.00202394]), 'test_accuracy': array([0.21428571, 0.20068027, 0.24829932, 0.2414966 , 0.21843003,
       0.2116041 , 0.18771331, 0.20819113, 0.27986348, 0.22866894]), 'test_qwkappa': array([0.39728286, 0.36535368, 0.49203011, 0.47098105, 0.43014349,
       0.41161763, 0.37080707, 0.37288367, 0.40451703, 0.48339454])}
Mean accuracy: 0.22392328945229972
Mean qwkappa: 0.41990111362235166


d:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
# random forest classification
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(rf, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

{'fit_time': array([3.26331758, 3.14728093, 3.05274773, 3.05057573, 2.88794446,
       2.93620825, 2.97148776, 2.97359633, 3.03866005, 3.01794744]), 'score_time': array([0.02467132, 0.02605987, 0.02403569, 0.02399993, 0.02299905,
       0.02400088, 0.02200031, 0.02199864, 0.02400184, 0.02410269]), 'test_accuracy': array([0.2414966 , 0.24829932, 0.28231293, 0.28911565, 0.24232082,
       0.24232082, 0.31740614, 0.25938567, 0.24914676, 0.27645051]), 'test_qwkappa': array([0.31101307, 0.39947035, 0.43605295, 0.4082973 , 0.37789486,
       0.32096598, 0.48178678, 0.37020308, 0.35820806, 0.3516736 ])}
Mean accuracy: 0.26482552065194676
Mean qwkappa: 0.3815566030411007


In [14]:
# support machine classification
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(clt, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

{'fit_time': array([30.18462658, 31.71394444, 32.52860308, 29.70220089, 31.0586617 ,
       31.11368322, 31.00566173, 28.450598  , 28.56923556, 28.38382649]), 'score_time': array([2.81480765, 3.39681315, 2.87610888, 2.86103988, 3.08868384,
       3.08585882, 2.79151297, 2.83535552, 2.78470159, 2.78787589]), 'test_accuracy': array([0.25170068, 0.23129252, 0.27891156, 0.2585034 , 0.24232082,
       0.2559727 , 0.25938567, 0.26962457, 0.3003413 , 0.26962457]), 'test_qwkappa': array([0.48737401, 0.44428811, 0.50995898, 0.5590035 , 0.45993222,
       0.45003855, 0.45914817, 0.57997671, 0.51428812, 0.46167048])}
Mean accuracy: 0.2617677787838685
Mean qwkappa: 0.49256788464136214
